In [ ]:
import numpy as np

# --- STEP 1: MY BIOMETRICS & EFFICIENCY ---
# I am tracking my power and pace relative to my Heart Rate.
# In my model, lower HR at higher output indicates better aerobic efficiency.
my_bio_metrics = {
    'weight_kg': 75,
    'ftp_watts': 255,
    'ftp_hr': 176,               # My HR at FTP effort
    'run_threshold_pace_sec': 231, # 3:51/km pace
    'run_threshold_hr': 182,      # My HR at running threshold pace
    'css_pace_100m_sec': 135,     # My 2:15/100m pace
    'css_hr': 124                 # My HR during ^ pace
}

# Calculated Features (Feature Engineering)
# I'm calculating my power-to-weight and my 'Heart Rate Efficiency'
my_wkg = my_bio_metrics['ftp_watts'] / my_bio_metrics['weight_kg']
bike_efficiency = my_bio_metrics['ftp_watts'] / my_bio_metrics['ftp_hr']

# --- STEP 2: MY TRAINING VOLUME (Weekly Averages) ---
# I am using my rounded averages as the 'Baseline'
avg_swim_m = 2300
avg_bike_z2_km = 257
avg_bike_th_km = 31
avg_run_z2_km = 27
avg_run_th_km = 7

# ENTER TOTAL WEEKS TRAINED HERE:
total_weeks_trained = 13

# Calculation of Cumulative Volume
# This creates the "Total Load" my body has adapted to over time.
cumulative_volume = np.array([
    avg_swim_m * total_weeks_trained,
    avg_bike_z2_km * total_weeks_trained,
    avg_bike_th_km * total_weeks_trained,
    avg_run_z2_km * total_weeks_trained,
    avg_run_th_km * total_weeks_trained
])

# --- STEP 3: VICTORIA 70.3 COURSE PHYSICS ---
# I've researched these multipliers to account for the specific terrain in Victoria.
victoria_course = {
    'swim_current': 1.0,    # Elk Lake (Neutral current)
    'bike_elevation': 1.12, # ~900m gain (12% difficulty penalty)
    'run_terrain': 1.05     # Hard-packed trail (5% surface penalty)
}

print(f"✅ My full profile is loaded: {my_wkg:.2f} W/kg and {bike_efficiency:.2f} Watts/BPM efficiency.")

In [ ]:
def Gabriels_lifecycle_predictor(total_data, bio, weeks, env):
    # 1. BASE BIAS: Starting point (minutes)
    # I am using 430 minutes as a starting point, this is if I did not train at all for this race.
    base_bias = 430

    # 2. These are my own custom shaving parametres (The 'W' parameters)
    # These are translated into per-unit (m/km) values.
    # Note: We divide by 'weeks' inside the logic to keep the shaving proportional to your average.
    w = {
        'swim': -0.0006,    # (0.6 mins / 1000m) (Time shaved off base time / milage I do)
        'bike_z2': -0.015, # (1.0 min / 66km)    (Here for EX I shave off 1 min for every 66km I bike in zone 2)
        'bike_th': -0.03,   # (.3 min / 10km)
        'run_z2': -0.1,    # (1 min / 10km)
        'run_th': -0.15     # (1.5 min / 10km)
    }

   # 3. MY EFFICIENCY FACTORS (The Biological Modifiers)

   # BIKE: Based on Power-to-Weight (3.0 is baseline)
    bike_wkg = bio['ftp_watts'] / bio['weight_kg']
    bike_eff = 1.0 + (bike_wkg - 3.0) * 0.1

    # SWIM: Based on HR at CSS Pace (140 bpm is baseline)
    # If my HR is 124, this factor becomes ~1.13 (13% more effective)
    swim_eff = 140 / bio['css_hr']

    # RUN: Based on Economy Ratio (Pace/HR), 1.25 is baseline for a fit individual.
    run_economy_ratio = bio['run_threshold_pace_sec'] / bio['run_threshold_hr']
    run_eff = 1.25 / run_economy_ratio

    # 4. Training Effect
    # I am taking the Total Volume and applying the shaving rules I came up with.
    # We include a 'Diminishing Returns' factor (log) because beginners and intermediate gain at different rates.
    # adds slightly less speed than week 1.
    adaptation_factor = np.log1p(weeks) / 2.3 # Andrew Ng principle: avoid linear scaling over long periods
    # This line models diminishing training gains over time and uses 2.3 to keep 13 weeks impactful but realistic.

    training_effect = (
        (total_data[0] * w['swim']) +
        (total_data[1] * w['bike_z2'] * bike_eff) +
        (total_data[2] * w['bike_th'] * bike_eff) +
        (total_data[3] * w['run_z2'] * run_eff) +
        (total_data[4] * w['run_th'] * run_eff)
    ) * adaptation_factor

    # 5. Course Physics & Final Math
    raw_time = base_bias + training_effect

    # Applying Victoria multi-pliers (Hills and Trail)
    swim_split = (raw_time * 0.15) * env['swim_current']
    bike_split = (raw_time * 0.50) * env['bike_elevation']
    run_split = (raw_time * 0.35) * env['run_terrain']

    total_time = swim_split + bike_split + run_split
    return total_time, (swim_split, bike_split, run_split)

# Run the model
final_time, splits = Gabriels_lifecycle_predictor(cumulative_volume, my_bio_metrics, total_weeks_trained, victoria_course)

h, m = int(final_time // 60), int(final_time % 60)
print(f"--- GABRIEL'S CUMULATIVE MODEL (Victoria 70.3) ---")
print(f"Predicted Finish: {h}h {m}m")
print(f"Goal: 4h 58m")
print(f"Total Base: Swim {cumulative_volume[0]}m | Bike {cumulative_volume[1]+cumulative_volume[2]}km | Run {cumulative_volume[3]+cumulative_volume[4]}km")